## Using all 8 cores of a v3 TPU using pytorch/XLA

## setup for pytorch/xla on TPU
thanks to this [kernel](https://www.kaggle.com/byrachonok/pytorch-xla-for-tpuv3)

In [ ]:
import os
import collections
from datetime import datetime, timedelta



## Imports

In [ ]:
import os
import re
import cv2
import time
import tensorflow
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image
import requests, threading
import matplotlib.pyplot as plt
from datetime import datetime, timedelta



import warnings
warnings.filterwarnings("ignore")


In [ ]:
os.listdir('../input')

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

## create classes dict

In [ ]:
import pickle
class_dict={}
inverted_class_dict={}
for i,c in enumerate(CLASSES):
    class_dict[c]=i
    inverted_class_dict[str(i)]=c
    



In [ ]:
with open('class_dict.pkl',mode='wb') as fout:
    pickle.dump(class_dict,fout)
with open('inverted_class_dict.pkl',mode='wb') as fout:
    pickle.dump(inverted_class_dict,fout)    

## create train dataframe

In [ ]:
img_ids=[]
labels=[]
for i,c in enumerate(CLASSES):
    #
    class_glob=glob(f'../input/104-flowers-garden-of-eden/jpeg-224x224/train/{c}/*')
    print(len(class_glob))
    for img_id in class_glob:
#         print(img_id)
        img_ids.append(img_id)
        labels.append(i)
#         break
#     break

print(len(img_ids),len(labels))    

In [ ]:
d = {'img_id': img_ids, 'label': labels}
train_df=pd.DataFrame(d)
train_df.head()
train_df.to_csv('train.csv',index=False)

# valid data

In [ ]:
img_ids=[]
labels=[]
for i,c in enumerate(CLASSES):
    #
    class_glob=glob(f'../input/104-flowers-garden-of-eden/jpeg-224x224/val/{c}/*')
    print(len(class_glob))
    for img_id in class_glob:
#         print(img_id)
        img_ids.append(img_id)
        labels.append(i)
#         break
#     break

print(len(img_ids),len(labels))    

In [ ]:
d = {'img_id': img_ids, 'label': labels}
train_df=pd.DataFrame(d)
train_df.head()
train_df.to_csv('valid.csv',index=False)

In [ ]:
test_jpeg = glob('../input/104-flowers-garden-of-eden/jpeg-224x224/test/*')
len(test_jpeg)

## Dataset